In [ ]:
import heapq

from math import sqrt
from typing import List
from collections import deque, Counter, defaultdict

import numpy as np

day 1: hit 61

In [ ]:
# 207. Course Schedule
class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        graph = defaultdict(list)
        for c,p in prerequisites: graph[c].append(p)
        completed = set()

        for idx_c in range(0, numCourses):
            if idx_c in completed: continue
            stack = [(idx_c, False)]
            reqs = set()

            while stack:
                curr_c, curr_c_done = stack.pop()
                if curr_c_done:
                    reqs.remove(curr_c)
                    completed.add(curr_c)
                if curr_c in completed: continue
                if curr_c in reqs: return False
                reqs.add(curr_c)
                stack.append((curr_c, True))
                for r in graph[curr_c]:
                    if r not in completed: stack.append((r, False))
        return True

In [ ]:
# 787. Cheapest Flights Within K Stops
class Solution:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
        graph = defaultdict(list)
        for f,t,p in flights: graph[f].append((t,p))

        best = dict()
        heap = [(0, src, 0)]
        while heap:
            cp, loc, steps = heapq.heappop(heap)
            if loc == dst: return cp

            if steps <= k:
                for nxt, p_here in graph[loc]:
                    new_steps = steps + 1
                    new_p = cp + p_here
                    if ((nxt, new_steps) not in best) or (new_p < best[(nxt, new_steps)]):
                        best[(nxt, new_steps)] = new_p
                        heapq.heappush(heap, (new_p, nxt, new_steps))

        return -1

In [ ]:
# 133. Clone Graph
# Definition for a Node.
class Node:
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

from typing import Optional
class Solution:
    def cloneGraph(self, node: Optional['Node']) -> Optional['Node']:
        if node is None: return None
        queue = deque([node])
        new_v = {node.val: Node(node.val)} # because values are unique
        
        while queue:
            curr = queue.popleft()
            
            for nei in curr.neighbors:
                if nei.val not in new_v:
                    queue.append(nei)
                    new_v[nei.val] = Node(nei.val) # Node(...) to make things into a node
                new_v[curr.val].neighbors.append(new_v[nei.val])
        
        return new_v[node.val]

In [ ]:
# 621. Task Scheduler
class Solution:
    def leastInterval(self, tasks: List[str], n: int) -> int:
        freqs = Counter(tasks)
        heap = []
        for f in freqs.values(): heap.append(-f)
        heapq.heapify(heap)
        t = 0
        queue = deque()

        while queue or heap:
            t += 1
            if heap:
                still = heapq.heappop(heap) + 1
                if still != 0: queue.append((still, t + n))
            
            if queue and queue[0][1] == t:
                heapq.heappush(heap, queue.popleft()[0])
        
        return t

day 2: hit 56

In [ ]:
# 347. Top K Frequent Elements
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        frequencies = Counter(nums)
        heap = []
        heapq.heapify(heap)
        for val, freq in frequencies.items():
            if len(heap) < k: heapq.heappush(heap, (freq, val))
            elif heap[0][0] < freq: heapq.heapreplace(heap, (freq, val))
        return [val for _,val in heap]

In [ ]:
# 973. K Closest Points to Origin
class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        heap = []
        for x, y in points:
            dist = -sqrt((x**2) + (y**2))
            if len(heap) == k: heapq.heappushpop(heap, (dist, x, y))
            else: heapq.heappush(heap, (dist,x,y))
        return [(x,y) for _,x,y in heap]

In [ ]:
# 215. Kth Largest Element in an Array
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        heap = nums[:k]
        heapq.heapify(heap)
        for n in nums[k:]:
            if n > heap[0]: heapq.heapreplace(heap, n)
        return heap[0]

In [ ]:
# 230. Kth Smallest Element in a BST
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def kthSmallest(self, root: Optional[TreeNode], k: int) -> int:
        if not root: return None
        curr = root
        stack = []
        idx = 0
        
        while curr or stack:
            while curr:
                stack.append(curr)
                curr = curr.left
            idx += 1
            curr = stack.pop()
            if idx == k: return curr.val
            curr = curr.right
        return curr.val

In [ ]:
# 450. Delete Node in a BST
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def recur(self, curr, key):
        if not curr: return None

        if key < curr.val: curr.left = self.recur(curr.left, key)
        elif key > curr.val: curr.right = self.recur(curr.right, key)
        else:
            # no child
            if not curr.left and not curr.right: return None

            # one child
            if not curr.left and curr.right: return curr.right
            if not curr.right and curr.left: return curr.left
            
            # both children
            nxt = curr.left
            while nxt.right: nxt = nxt.right
            curr.val = nxt.val
            curr.left = self.recur(curr.left, nxt.val)

        return curr
    
    def deleteNode(self, root: Optional[TreeNode], key: int) -> Optional[TreeNode]:
        return self.recur(root, key)

day 3: hit 46

In [ ]:
# 1382. Balance a Binary Search Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def conv_to_arr(self, root):
        arr = []
        stack = []
        while stack or root:
            while root:
                stack.append(root)
                root = root.left
            root = stack.pop()
            arr.append(root.val)
            root = root.right
        return arr

    def recurr(self, arr):
        if not arr: return None
        idx = int(len(arr) / 2)
        node = TreeNode(arr[idx])
        node.left = self.recurr(arr[0:idx])
        node.right = self.recurr(arr[idx + 1:])
        return node
        
    def balanceBST(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if root is None: return None
        arr = self.conv_to_arr(root)
        return self.recurr(arr)

In [ ]:
# 530. Minimum Absolute Difference in BST
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def getMinimumDifference(self, root: Optional[TreeNode]) -> int:
        if root is None: return None

        mad = float('inf')
        stack = []
        first = None
        while stack or root:
            while root:
                stack.append(root)
                root = root.left
            root = stack.pop()
            if first is not None: mad = min(mad, abs(first - root.val))
            first = root.val
            root = root.right
        return mad

In [ ]:
# 235. Lowest Common Ancestor of a Binary Search Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        if root is None: return None
        sm = min(p.val, q.val)
        bi = max(p.val, q.val)
        while root:
            if sm < root.val and bi < root.val: root = root.left
            elif sm > root.val and bi > root.val: root = root.right
            else: return root

In [ ]:
# 653. Two Sum IV - Input is a BST
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def findTarget(self, root: Optional[TreeNode], k: int) -> bool:
        if root is None: return True
        done = set()
        stack = [root]
        
        while stack:
            curr = stack.pop()
            diff = k - curr.val
            if diff in done: return True
            done.add(curr.val)
            if curr.left: stack.append(curr.left)
            if curr.right: stack.append(curr.right)
        return False

In [ ]:
# 108. Convert Sorted Array to Binary Search Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def former(self, nums):
        if not nums: return None
        mid = int(len(nums) / 2)
        node = TreeNode(nums[mid])
        node.left = self.former(nums[:mid])
        node.right = self.former(nums[mid + 1:])
        return node

    def sortedArrayToBST(self, nums: List[int]) -> Optional[TreeNode]:
        return self.former(nums)

In [ ]:
# 701. Insert into a Binary Search Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:
    def insertIntoBST(self, root: Optional[TreeNode], val: int) -> Optional[TreeNode]:
        if not root: return TreeNode(val)
        curr = root
        while True:
            if curr.val < val:
                if curr.right: curr = curr.right
                else:
                    curr.right = TreeNode(val)
                    break
            else:
                if curr.left: curr = curr.left
                else:
                    curr.left = TreeNode(val)
                    break
        return root

In [ ]:
# 700. Search in a Binary Search Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:
    def searchBST(self, root: Optional[TreeNode], val: int) -> Optional[TreeNode]:
        curr = root
        while curr:
            if curr.val == val: return curr
            elif curr.val > val: curr = curr.left
            else: curr = curr.right
        return None

In [ ]:
# 236. Lowest Common Ancestor of a Binary Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        if not root: return None
        d = {root: None}
        queue = deque([root])

        while queue:
            curr = queue.popleft()
            if curr.left:
                d[curr.left] = curr
                queue.append(curr.left)
            if curr.right:
                d[curr.right] = curr
                queue.append(curr.right)
        
        top_ans = set()
        while p:
            top_ans.add(p)
            p = d[p]
        
        while q:
            if q in top_ans: return q
            q = d[q]      
        return root

In [ ]:
# 226. Invert Binary Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:
    def invertTree(self, root: Optional[TreeNode]) -> Optional[TreeNode]:
        if not root: return None
        queue = deque([root])
        while queue:
            curr = queue.popleft()
            curr.left, curr.right = curr.right, curr.left
            if curr.left: queue.append(curr.left)
            if curr.right: queue.append(curr.right)
        
        return root

In [ ]:
# 543. Diameter of Binary Tree
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:
    def __init__(self):
        self.dia = 0

    def incrementer(self, root):
        if not root: return 0
        lft = self.incrementer(root.left)
        rgt = self.incrementer(root.right)
        self.dia = max(self.dia, lft + rgt)
        return 1 + max(lft, rgt)
    
    def diameterOfBinaryTree(self, root: Optional[TreeNode]) -> int:
        self.incrementer(root)
        return self.dia

day 4: hit 31

day 5: hit 1